<a href="https://colab.research.google.com/github/EgorSolovei/VK-recommendation-system/blob/main/create_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
! pip install sketch

In [2]:
import pandas as pd
import numpy as np
import sketch
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Data download

In [3]:
# файл, который обрабывается в NN_recomd. Если дождаться конца обработки будет примерно 3Гб, а так ...мб
user_movie = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ML_&_AI/Pratice/NN/VK recomd/data/user_movie.csv')
user_movie.drop(columns='Unnamed: 0', inplace=True)
user_movie.head()

,userId,1,2,5,6,10,11,16,19,20,...,47868,103449,69498,130044,819,6499,166149,6588,87549,4955
0,4,4.0,4.0,2.0,4.5,4.0,3.5,4.0,2.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# файл до обработки в NN_recomd (очищенный ratings). Примерно 600Мб
user_movie_raw = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ML_&_AI/Pratice/NN/VK recomd/data/user_movie_raw.csv')
user_movie_raw.drop(columns='Unnamed: 0', inplace=True)
user_movie_raw.head()

,userId,movieId,rating
0,4,1,4.0
1,4,2,4.0
2,4,5,2.0
3,4,6,4.5
4,4,10,4.0


In [5]:
genome_scores = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ML_&_AI/Pratice/NN/VK recomd/data/genome-scores.csv')
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02900
1,1,2,0.02375
2,1,3,0.05425
3,1,4,0.06875
4,1,5,0.16000


In [6]:
genome_tags = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ML_&_AI/Pratice/NN/VK recomd/data/genome-tags.csv')
# links = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ML_&_AI/Pratice/NN/VK recomd/data/links.csv') # вряд ли буду использовать
movies = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ML_&_AI/Pratice/NN/VK recomd/data/movies.csv')
tags = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ML_&_AI/Pratice/NN/VK recomd/data/tags.csv')

In [7]:
frames_and_names = { "user_movie_raw": user_movie_raw, 'user_movie': user_movie, 
                     "genome_scores": genome_scores, "genome_tags": genome_tags, 
                     "movies": movies, "tags": tags}

for name, data in frames_and_names.items():
    print(f"Файл {name} имеет размер {data.shape}")
    print(f"Коллонки: {list(data.columns)} \n")

Файл user_movie_raw имеет размер (25499629, 3)
Коллонки: ['userId', 'movieId', 'rating'] 

Файл user_movie имеет размер (30029, 13628)
Коллонки: ['userId', '1', '2', '5', '6', '10', '11', '16', '19', '20', '23', '25', '31', '32', '34', '36', '39', '41', '44', '45', '47', '50', '61', '62', '65', '66', '70', '76', '85', '95', '100', '104', '110', '141', '145', '150', '151', '153', '158', '160', '161', '163', '165', '169', '170', '172', '173', '175', '180', '181', '185', '186', '193', '198', '204', '216', '223', '231', '253', '255', '256', '260', '267', '288', '292', '293', '296', '315', '316', '317', '318', '327', '329', '332', '333', '338', '339', '344', '349', '350', '353', '356', '361', '364', '367', '368', '370', '377', '379', '380', '382', '405', '410', '420', '423', '432', '434', '435', '440', '442', '454', '455', '457', '466', '471', '474', '480', '481', '485', '494', '500', '514', '517', '520', '524', '527', '539', '541', '546', '552', '586', '587', '588', '589', '590', '592', '5

## Feature's users

In [9]:
user_movie.userId.unique().shape

(5566,)

In [9]:
summ = user_movie.drop(columns='userId').sum(axis=1) # сумма всех оценок каждого пользователя
summ

0        2500.5
1         307.0
2         149.0
3         104.0
4         507.0
          ...  
30024     271.0
30025    6202.5
30026     725.0
30027     114.0
30028     176.0
Length: 30029, dtype: float64

In [11]:
summ = user_movie_raw.groupby('userId')['rating'].sum()
summ

userId
4         2500.5
5          307.0
6          149.0
8          104.0
10         507.0
           ...  
283219     328.0
283221     129.0
283222     240.0
283224    1228.0
283228     490.5
Name: rating, Length: 143545, dtype: float64

In [13]:
# признак mean_score, quan_cnt сделаем из файла user_movie_raw
# user_movie_raw.sketch.howto('count rating for every userId ')

In [12]:
user_rating_count = user_movie_raw.groupby('userId')['rating'].count()
user_rating_count

userId
4         736
5          72
6          42
8          31
10        121
         ... 
283219     81
283221     35
283222     69
283224    329
283228    113
Name: rating, Length: 143545, dtype: int64

In [13]:
# среднее значение оценки для каждого пользователя
mea_score = user_movie_raw.groupby('userId')['rating'].mean()

In [28]:
user_movie_raw.userId.unique()

(143545,)

In [ ]:
# Create a list of movieIds from user_movie
user_movie_ids = user_movie_raw['movieId'].unique().tolist()  
user_movie_ids

In [25]:
# Create a list of movieIds from user_movie
user_movie_ids = user_movie_raw['movieId'].unique().tolist()  

# Filter the genome_scores dataframe to only include movieIds in the user_movie_raw list
movies = movies[genome_scores['movieId'].isin(user_movie_ids)]
movies

<ipython-input-25-cc634af51ef9>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  movies = movies[genome_scores['movieId'].isin(user_movie_ids)]


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed)
58094,193878,Les tribulations d'une caissière (2011),Comedy
58095,193880,Her Name Was Mumu (2016),Drama
58096,193882,Flora (2017),Adventure|Drama|Horror|Sci-Fi


In [ ]:
# movies.sketch.howto('In column == genres split values by "|" and ')

In [ ]:
ser = pd.Series(list(movies['genres'].str.split('|'))) # серия жанров (выбрать только нужные MovieId из )
ser

0        [Adventure, Animation, Children, Comedy, Fantasy]
1                           [Adventure, Children, Fantasy]
2                                        [Comedy, Romance]
3                                 [Comedy, Drama, Romance]
4                                                 [Comedy]
                               ...                        
58093                                 [(no genres listed)]
58094                                             [Comedy]
58095                                              [Drama]
58096                   [Adventure, Drama, Horror, Sci-Fi]
58097                               [Action, Crime, Drama]
Length: 58098, dtype: object

In [ ]:
ser.values[0]

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']

In [ ]:
all_genres = set()
for 
for genre in ser.values:


## Feature's movies

In [ ]:
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [ ]:
genome_scores.shape

(14862528, 3)

In [ ]:
# genome_scores.sketch.howto('Create new column "tag" and put tag from "genome_tags" by key "tagId"')

In [ ]:
# merge the two dataframes on tagId
genome_scores = genome_scores.merge(genome_tags, on='tagId', how='left')

# create new column "tag" and put tag from "genome_tags" by key "tagId"
genome_scores['tag'] = genome_scores['tag'].fillna('')
genome_scores.head()

,movieId,tagId,relevance,tag
0,1,1,0.02900,007
1,1,2,0.02375,007 (series)
2,1,3,0.05425,18th century
3,1,4,0.06875,1920s
4,1,5,0.16000,1930s


In [ ]:
# genome_scores.sketch.howto("In 'movieId' delete if not in user_movie.movieId")

Расскоментировать, когда будет файл user_movie_raw

In [ ]:
# # Create a list of movieIds from user_movie
# user_movie_ids = user_movie_raw['movieId'].unique().tolist()  

# # Filter the genome_scores dataframe to only include movieIds in the user_movie_raw list
# genome_scores = genome_scores[genome_scores['movieId'].isin(user_movie_ids)]

,movieId,tagId,relevance,tag
0,1,1,0.02900,007
1,1,2,0.02375,007 (series)
2,1,3,0.05425,18th century
3,1,4,0.06875,1920s
4,1,5,0.16000,1930s
...,...,...,...,...
14862523,187595,1124,0.10700,writing
14862524,187595,1125,0.05850,wuxia
14862525,187595,1126,0.03800,wwii
14862526,187595,1127,0.10225,zombie


In [ ]:
genome_scores.sketch.howto('Get max relevance for every movieId')

In [ ]:
# это максимальное значение. Можно сделать ещё 2, 3 и тд по релевантности тег
genome_scores.groupby('movieId')['relevance'].max()

movieId
1         0.99950
2         0.97800
3         0.97000
4         0.97625
5         0.96925
           ...   
185435    0.88750
185585    0.76700
186587    0.91875
187593    0.85325
187595    0.88225
Name: relevance, Length: 13156, dtype: float64

In [ ]:
genome_scores.query("movieId == 1").relevance.sort_values(ascending=False).head(20)

1035    0.99950
243     0.99900
785     0.99550
63      0.98875
588     0.98850
587     0.97850
784     0.96975
185     0.95850
62      0.95500
203     0.95450
411     0.93350
535     0.92875
970     0.91700
28      0.90700
202     0.90550
468     0.90400
602     0.88475
741     0.88225
1061    0.88200
309     0.87000
Name: relevance, dtype: float64

## Summary

In [16]:
user_movie_raw.userId.values.shape, mea_score.shape, user_rating_count.shape, summ.shape

((25499629,), (143545,), (143545,), (143545,))

In [ ]:
user_ftr = pd.DataFrame({
    'userId': user_movie.userId.values,
    'mean_score': mea_score,
    'quantity': user_rating_count,
    'summ': summ
    })

In [ ]:
movie_ftr = pd.DataFrame({
    # 'movieId': user_movie_raw.movieId.unique().values, # раскоментить, когда будет доступен user_movie_raw
    'average_mark': [],

    })